In [2]:
import h5py
import fsspec
URL = "https://zenodo.org/records/10119316/files/val.h5"
remote_f = fsspec.open(URL, mode="rb")
if hasattr(remote_f, "open"):
    remote_f = remote_f.open()
f = h5py.File(remote_f)
f.visititems(print)

data <HDF5 dataset "data": shape (25934, 256, 144), type "<f4">
target <HDF5 dataset "target": shape (25934,), type "|i1">
